In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

<b>Check for missing values in training dataset</b>

In [2]:
data = pd.read_csv("train.csv")
data.head(10)

#check for missing values in dataset
data.isnull().sum()
#data.columns

Unnamed: 0                    0
ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

<ul><li><b> One Hot Encoding for converting categorical variables to numeric </b> </li>
<li><b> Pay Amount which has negative value is converted to 0 </b> </li>
    <li><b> Checking for data imbalance in dataset with respect to default payment next month </b></li> </ul>

In [3]:
data.rename(columns=lambda col:col.lower(),inplace=True)
data['grad_school']=(data['education']==1).astype('int')
data['university']=(data['education']==2).astype('int')
data['highschool']=(data['education']==3).astype('int')
data.drop('education',axis=1,inplace=True)
data.drop('unnamed: 0',axis=1,inplace=True)
data.drop('id',axis=1,inplace=True)
data['male']=(data['sex']==1).astype('int')
data.drop('sex',axis=1,inplace=True)
data['married']=(data['marriage']==1).astype('int')
data.drop('marriage',axis=1,inplace=True)
pay_amt=['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6']
for pay in pay_amt:
    data.loc[data[pay]<=0,pay]=0   
data.rename(columns={'default payment next month':'defaulter'},inplace=True)

# Checking for data imbalance wrt Class Labels
data.defaulter.value_counts()

0    18890
1     5279
Name: defaulter, dtype: int64

<b> Scaling attribute values of Training Data Set and Feature Selection of important attributes through correlation </b>

In [4]:
cols=data.columns.values.tolist()
num_cols=['limit_bal','bill_amt1',
 'bill_amt2',
 'bill_amt3',
 'bill_amt4',
 'bill_amt5',
 'bill_amt6',
 'pay_amt1',
 'pay_amt2',
 'pay_amt3',
 'pay_amt4',
 'pay_amt5',
 'pay_amt6']
cat_cols=set(cols)-set(num_cols)
if 'defaulter' in cat_cols:
    cat_cols.remove('defaulter')
cat_cols
data['pay_amt1']=data['pay_amt1']/data['bill_amt1']
data['pay_amt2']=data['pay_amt2']/data['bill_amt2']
data['pay_amt3']=data['pay_amt3']/data['bill_amt3']
data['pay_amt4']=data['pay_amt4']/data['bill_amt4']
data['pay_amt5']=data['pay_amt5']/data['bill_amt5']
data['pay_amt6']=data['pay_amt6']/data['bill_amt6']
data = data.replace({None:0, np.inf:1})
data['bill_amt1']=data['bill_amt1']/data['limit_bal']
data['bill_amt2']=data['bill_amt2']/data['limit_bal']
data['bill_amt3']=data['bill_amt3']/data['limit_bal']
data['bill_amt4']=data['bill_amt4']/data['limit_bal']
data['bill_amt5']=data['bill_amt5']/data['limit_bal']
data['bill_amt6']=data['bill_amt6']/data['limit_bal']
data['limit_bal']=data['limit_bal']/data['limit_bal'].max()
data=data[['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6','bill_amt1','limit_bal','pay_amt2','pay_amt1','age','grad_school','university','highschool','defaulter']]

<b> Using Chi square independence test to check dependency of attribute value (Here Age) with class label(defaulter) </b> <br>
<ul> <li> Null Hypothesis H<sub>0</sub>:There is no statistically significant relationship between age attribute and defaulter class label.</li>
<li> Alternate Hypothesis H<sub>a</sub>:There is statistically significant relationship between age attribute and defaulter class label.</li></ul>

In [5]:
#data.columns
age_by_defaulter=data[['age','defaulter']]
contingency_table = pd.crosstab(age_by_defaulter['defaulter'],
    age_by_defaulter['age'],
    margins = True)
contingency_table

age        21   22   23   24   25    26    27    28    29    30  ...  67  68  \
defaulter                                                        ...           
0          42  317  545  677  714   804   920   907  1053   872  ...  10   2   
1          11  135  209  236  247   198   246   229   255   212  ...   3   1   
All        53  452  754  913  961  1002  1166  1136  1308  1084  ...  13   3   

age        69  70  71  72  73  74  75    All  
defaulter                                     
0          10   5   3   1   1   1   1  18890  
1           3   1   0   0   3   0   1   5279  
All        13   6   3   1   4   1   2  24169  

[3 rows x 56 columns]

In [6]:
data_obs = np.array([contingency_table.iloc[0][0:55].values,
                  contingency_table.iloc[1][0:55].values])
stats.chi2_contingency(data_obs)[0:3]

(148.36158147153245, 9.738846011833446e-11, 54)

<b> The first value is the chi-square test statistic while the second and third value point to the p value and degrees of freedom(number of attributes-1)*(number of rows-1) respectively. <br>
<b> Here we reject the null hypothesis that is age is independent wrt defaulter class label with p-value less than 0.05. Thus age and defaulter column have some sort of relationship with each other. </b>

<br>

<b> Using Chi square independence test to check dependency of attribute value (Here Limit Balance) with class label(defaulter) </b> <br>
<ul> <li> Null Hypothesis H<sub>0</sub>:There is no statistically significant relationship between Limit Balance attribute and defaulter class label.</li>
<li> Alternate Hypothesis H<sub>a</sub>:There is statistically significant relationship between Limit Balance attribute and defaulter class label.</li></ul>

In [7]:
limitbal_by_defaulter=data[['limit_bal','defaulter']]
contingency_table = pd.crosstab(limitbal_by_defaulter['defaulter'],
    limitbal_by_defaulter['limit_bal'],
    margins = True)
contingency_table

limit_bal  0.0125  0.02  0.025  0.0375  0.05  0.0625  0.075  0.0875   0.1  \
defaulter                                                                   
0             237     1   1024     826   109    2021    485     420   989   
1             161     0    565     441    78     712    188     164   295   
All           398     1   1589    1267   187    2733    673     584  1284   

limit_bal  0.1125  ...  0.8625  0.875  0.8875  0.9  0.9125  0.925  0.9375  \
defaulter          ...                                                      
0             396  ...       1      6       5    2       2      0       4   
1             128  ...       0      0       1    1       0      1       0   
All           524  ...       1      6       6    3       2      1       4   

limit_bal  0.95  1.0    All  
defaulter                    
0             1    2  18890  
1             0    0   5279  
All           1    2  24169  

[3 rows x 80 columns]

In [8]:
data_obs = np.array([contingency_table.iloc[0][0:79].values,
                  contingency_table.iloc[1][0:79].values])
stats.chi2_contingency(data_obs)[0:3]

(856.1592981208651, 2.5529816345174594e-131, 78)

<b> The first value is the chi-square test statistic while the second and third value point to the p value and degrees of freedom(number of attributes-1)*(number of rows-1) respectively. <br>
<b> Here we reject the null hypothesis that is limit balance is independent wrt defaulter class label with p-value less than 0.05. Thus limit balance and defaulter column have some sort of relationship with each other. </b>
<br>

In [9]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,StratifiedKFold
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV,RFE
from sklearn.metrics import SCORERS,accuracy_score,precision_score,f1_score,recall_score,confusion_matrix,roc_curve, auc, classification_report
from sklearn.preprocessing import RobustScaler

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


<b> Setting up Training Dataset values to pass in model </b>

In [10]:
target='defaulter'
x=data.drop('defaulter',axis=1)
y=data[target]

<b> Using train test split to get train and validation datasets for model accuracy. </b> <br>
<b> Using SMOTE for handling imbalanced data(Here train values are taken).</b>

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=0,stratify=y)
sm = SMOTE(random_state=1, ratio='auto',kind='regular')
x_train, y_train = sm.fit_sample(x_train, y_train)

<b>Using GridSearch CV for tuning hyperparmeters for the given learning model</b>
<br> <b> Here scoring criterion is taken as roc_auc as here recall is maximised but it should be with the handling of recall-precision tradeoff.
<b> Recall is maximised here because here according to business more False Negatives incur more penalty as a defaulter will be classified as a person who has paid and not defaulted.

In [12]:
def Grid_Search(model, parameters):
    gs = GridSearchCV(model, parameters, scoring='roc_auc',cv=3, n_jobs=-1)
    gs.fit(x_train, y_train)
    print('Best params: ', gs.best_params_)
    print('Best auc on training set: ', gs.best_score_)
    print('Best auc on test set: ', gs.score(x_test, y_test))
    return gs.predict(x_test), gs.decision_function(x_test)

<b>Using Logistic Regression Model with liblinear solver</b>

In [13]:
logreg = LogisticRegression(solver='liblinear')
logreg_params = {'C': [0.001, 0.01, 0.1, 1, 10], 'class_weight': [None, 'balanced'], 'penalty': ['l1', 'l2']}
logreg_pred, logreg_prob = Grid_Search(logreg, logreg_params)

Best params:  {'C': 0.1, 'class_weight': None, 'penalty': 'l2'}
Best auc on training set:  0.7615101454047604
Best auc on test set:  0.7461624448436375


<b>Using Recursive Feature Elimination for getting important features for model fitting</b>
<br><b>The scoring criterion used here is roc_auc as here we would be maximising recall.</b>

In [14]:
lr = LogisticRegression(penalty='l2', C=0.1, solver='liblinear', class_weight=None)
rfecv = RFECV(estimator=lr, scoring='roc_auc',step=3,cv=3)
model = rfecv.fit(x_train, y_train)
lr_pred = model.predict(x_test)
lr_prob = model.decision_function(x_test)

<ul><li><b> Predicting probability of predictions made by the model.
    <li><b> Getting the classification threshold probability based on evaluation metric.
</ul>

In [15]:
pred_proba_df = pd.DataFrame(model.predict_proba(x_test))
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>i else 0)
    test_accuracy = accuracy_score(y_test.as_matrix().reshape(y_test.as_matrix().size,1),
                                           y_test_pred.iloc[:,1].as_matrix().reshape(y_test_pred.iloc[:,1].as_matrix().size,1))
    print('Our testing accuracy is {}'.format(test_accuracy))

    print(confusion_matrix(y_test.as_matrix().reshape(y_test.as_matrix().size,1),
                           y_test_pred.iloc[:,1].as_matrix().reshape(y_test_pred.iloc[:,1].as_matrix().size,1)))
    



******** For i = 0.05 ******
Our testing accuracy is 0.21842250413678985
[[   0 2834]
 [   0  792]]

******** For i = 0.1 ******
Our testing accuracy is 0.21869829012686157
[[   1 2833]
 [   0  792]]

******** For i = 0.15 ******
Our testing accuracy is 0.22007722007722008
[[   6 2828]
 [   0  792]]

******** For i = 0.2 ******
Our testing accuracy is 0.2289023717595146
[[  40 2794]
 [   2  790]]

******** For i = 0.25 ******
Our testing accuracy is 0.2713734142305571
[[ 220 2614]
 [  28  764]]

******** For i = 0.3 ******
Our testing accuracy is 0.361003861003861
[[ 581 2253]
 [  64  728]]

******** For i = 0.35 ******
Our testing accuracy is 0.5220628792057364
[[1236 1598]
 [ 135  657]]

******** For i = 0.4 ******
Our testing accuracy is 0.6889134031991175
[[1966  868]
 [ 260  532]]

******** For i = 0.45 ******
Our testing accuracy is 0.7595146166574738
[[2289  545]
 [ 327  465]]

******** For i = 0.5 ******
Our testing accuracy is 0.7777164920022063
[[2386  448]
 [ 358  434]]

**

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


Our testing accuracy is 0.819911748483177
[[2618  216]
 [ 437  355]]

******** For i = 0.7 ******
Our testing accuracy is 0.8257032542746828
[[2655  179]
 [ 453  339]]

******** For i = 0.75 ******
Our testing accuracy is 0.826530612244898
[[2687  147]
 [ 482  310]]

******** For i = 0.8 ******
Our testing accuracy is 0.8229453943739659
[[2713  121]
 [ 521  271]]

******** For i = 0.85 ******
Our testing accuracy is 0.8127413127413128
[[2765   69]
 [ 610  182]]

******** For i = 0.9 ******
Our testing accuracy is 0.8047435190292334
[[2797   37]
 [ 671  121]]

******** For i = 0.95 ******
Our testing accuracy is 0.7879205736348593
[[2820   14]
 [ 755   37]]

******** For i = 0.99 ******
Our testing accuracy is 0.7829564258135687
[[2828    6]
 [ 781   11]]


<b> Displaying Classification report of various metrics of the given model.

In [16]:
y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>=0.45 else 0) 
print(classification_report(y_test, y_test_pred.iloc[:,1]))

             precision    recall  f1-score   support

          0       0.88      0.81      0.84      2834
          1       0.46      0.59      0.52       792

avg / total       0.78      0.76      0.77      3626



<b> Calculating Accuracy Score of training and validation set

In [17]:
print('accuracy score on training set: ', cross_val_score(lr, x_train, y_train, n_jobs=-1,scoring='roc_auc',cv=3).mean())
print('accuracy score on validation set: ', accuracy_score(y_test_pred.iloc[:,1], y_test))

accuracy score on training set:  0.7615101454047606
accuracy score on validation set:  0.7595146166574738


<b>Printing Confusion Matrix for given model.
<br> <b> Relationship of the predicted values of defaulter and not defaulter with Actual values can be established.

In [18]:
conmat = np.array(confusion_matrix(y_test, y_test_pred.iloc[:,1], labels=[1,0]))
confusion = pd.DataFrame(conmat, index=['defaulter', 'not defaulter'], 
                             columns=['predicted defaulter', 'predicted not defaulter'])
print(confusion)

               predicted defaulter  predicted not defaulter
defaulter                      465                      327
not defaulter                  545                     2289


<b> Get the scoring criterion from sklearn.metrics

In [19]:
print(sorted(SCORERS.keys()))

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'log_loss', 'mean_absolute_error', 'mean_squared_error', 'median_absolute_error', 'mutual_info_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'v_measure_score']


<h3><b> Predicting Defaulters with test set based on above Logistic Regression Model

<b>Check for missing values in test dataset

In [20]:
test_data_orig = pd.read_csv("test.csv")
test_data = pd.read_csv("test.csv")
#test_data.head(10)

#check for missing values in dataset
test_data.isnull().sum()
#data.columns

Unnamed: 0    0
ID            0
LIMIT_BAL     0
SEX           0
EDUCATION     0
MARRIAGE      0
AGE           0
PAY_0         0
PAY_2         0
PAY_3         0
PAY_4         0
PAY_5         0
PAY_6         0
BILL_AMT1     0
BILL_AMT2     0
BILL_AMT3     0
BILL_AMT4     0
BILL_AMT5     0
BILL_AMT6     0
PAY_AMT1      0
PAY_AMT2      0
PAY_AMT3      0
PAY_AMT4      0
PAY_AMT5      0
PAY_AMT6      0
dtype: int64

<ul><li><b> One Hot Encoding for converting categorical variables to numeric </b> </li>
<li><b> Pay Amount which has negative value is converted to 0 </b> </li> </ul>

In [21]:
test_data.rename(columns=lambda col:col.lower(),inplace=True)
test_data['grad_school']=(test_data['education']==1).astype('int')
test_data['university']=(test_data['education']==2).astype('int')
test_data['highschool']=(test_data['education']==3).astype('int')
test_data.drop('education',axis=1,inplace=True)
test_data.drop('id',axis=1,inplace=True)
test_data.drop('unnamed: 0',axis=1,inplace=True)
test_data['male']=(test_data['sex']==1).astype('int')
test_data.drop('sex',axis=1,inplace=True)
test_data['married']=(test_data['marriage']==1).astype('int')
test_data.drop('marriage',axis=1,inplace=True)
pay_amt=['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6']
for pay in pay_amt:
    test_data.loc[test_data[pay]<=0,pay]=0   
#test_data.rename(columns={'default payment next month':'defaulter'},inplace=True)

<b> Scaling attribute values of Test Data Set and Feature Selection of important attributes through correlation </b>

In [22]:
cols=test_data.columns.values.tolist()
num_cols=['limit_bal','bill_amt1',
 'bill_amt2',
 'bill_amt3',
 'bill_amt4',
 'bill_amt5',
 'bill_amt6',
 'pay_amt1',
 'pay_amt2',
 'pay_amt3',
 'pay_amt4',
 'pay_amt5',
 'pay_amt6']
cat_cols=set(cols)-set(num_cols)
cat_cols
test_data['pay_amt1']=test_data['pay_amt1']/test_data['bill_amt1']
test_data['pay_amt2']=test_data['pay_amt2']/test_data['bill_amt2']
test_data['pay_amt3']=test_data['pay_amt3']/test_data['bill_amt3']
test_data['pay_amt4']=test_data['pay_amt4']/test_data['bill_amt4']
test_data['pay_amt5']=test_data['pay_amt5']/test_data['bill_amt5']
test_data['pay_amt6']=test_data['pay_amt6']/test_data['bill_amt6']
test_data = test_data.replace({None:0, np.inf:1})
test_data['bill_amt1']=test_data['bill_amt1']/test_data['limit_bal']
test_data['bill_amt2']=test_data['bill_amt2']/test_data['limit_bal']
test_data['bill_amt3']=test_data['bill_amt3']/test_data['limit_bal']
test_data['bill_amt4']=test_data['bill_amt4']/test_data['limit_bal']
test_data['bill_amt5']=test_data['bill_amt5']/test_data['limit_bal']
test_data['bill_amt6']=test_data['bill_amt6']/test_data['limit_bal']
test_data['limit_bal']=test_data['limit_bal']/test_data['limit_bal'].max()
test_data=test_data[['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6','bill_amt1','limit_bal','pay_amt2','pay_amt1','age','grad_school','university','highschool']]

<b> Predicting defaulter from test data by using the above model and saving it. </b>

In [23]:
pred_testdataproba_df = pd.DataFrame(model.predict_proba(test_data))
y_testdata_pred = pred_testdataproba_df.applymap(lambda x: 1 if x>=0.45 else 0)
test_data_orig.loc[:,'Prediction']=y_testdata_pred.iloc[:,1]
test_data_orig.to_csv('test.csv')